In [9]:
from audioop import avg
from statistics import mean
import pandas as pd
import os
os.chdir(r'C:\Users\brand\Desktop\QuantumCashUsageTaskFiles')
UploadFilePath = os.getcwd()

OutputFileName = 'QuantumtumTetheredUsage_Dispense_v4.xlsx' #Requires XLSX file type

raw = pd.read_csv('QuantumFinTx.csv')
raw.calendardate = pd.to_datetime(raw['CalendarDate'])


UsageByDay = raw.groupby(['Name',
                          'Number',
                          'MachineID',
                          'WeekNum',
                          'CalendarDate']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()


SumWeekUsage = UsageByDay.groupby(['Name',
                              'Number',
                              'WeekNum']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()

# Max and Avg are done using the Sum By Day df

MaxByWeek = UsageByDay.groupby(['Name',
                              'Number',
                              'MachineID',
                              'WeekNum']).agg({
                              '0.01c':   min,
                              '0.05c':   min,
                              '0.10c':   min,
                              '0.25c':   min,
                              '1.00n':   min,
                              '5.00n':   min,
                              '10.00n':  min,
                              '20.00n':  min,
                              '50.00n':  min,
                              '100.00n': min
                          }).reset_index()


            
AvgByWeek = UsageByDay.groupby(['Name',
                                'Number',
                                'MachineID',
                                'WeekNum']).agg({
                                '0.01c':   mean,
                                '0.05c':   mean,
                                '0.10c':   mean,
                                '0.25c':   mean,
                                '1.00n':   mean,
                                '5.00n':   mean,
                                '10.00n':  mean,
                                '20.00n':  mean,
                                '50.00n':  mean,
                                '100.00n': mean
                            }).reset_index()

Dispense = raw[raw['ChildTran'] == 'CashDispensed']

DispenseByDay = Dispense.groupby(['Name',
                          'Number',
                          'MachineID',
                          'WeekNum',
                          'CalendarDate']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()

DispenseWeeklyMax = DispenseByDay.groupby(['Name',
                                      'Number',
                                      'MachineID',
                                      'WeekNum']).agg({
                                          '0.01c':   min,
                                          '0.05c':   min,
                                          '0.10c':   min,
                                          '0.25c':   min,
                                          '1.00n':   min,
                                          '5.00n':   min,
                                          '10.00n':  min,
                                          '20.00n':  min,
                                          '50.00n':  min,
                                          '100.00n': min
                                      }).reset_index()

DispenseWeeklyAvg = DispenseByDay.groupby(['Name',
                                      'Number',
                                      'MachineID',
                                      'WeekNum']).agg({
                                           '0.01c':   mean,
                                           '0.05c':   mean,
                                           '0.10c':   mean,
                                           '0.25c':   mean,
                                           '1.00n':   mean,
                                           '5.00n':   mean,
                                           '10.00n':  mean,
                                           '20.00n':  mean,
                                           '50.00n':  mean,
                                           '100.00n': mean
                                       }).reset_index()

SumWeekDispense = DispenseByDay.groupby(['Name',
                              'Number',
                              'WeekNum']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()




#Format and Merge Dataframes
Week = pd.DataFrame()                                               #Create DataFrame to store Week Number
Week[['WeekNum','CalendarDate']] = raw[['WeekNum','CalendarDate']]  # and associated week of date

DateMap = Week.groupby('WeekNum')['CalendarDate'].min()

UsageMerge = MaxByWeek.merge(AvgByWeek
                            ,how='inner'
                            ,on=['Name','Number','MachineID','WeekNum']
                            ,suffixes=('_max','_avg'))

UsageMerge.insert(4,
                 'WeekOfDate',
                 UsageMerge['WeekNum'].map(DateMap.to_dict()))

DispenseMerge  = DispenseWeeklyMax.merge(DispenseWeeklyAvg
                                        ,how='inner'
                                        ,on=['Name','Number','MachineID','WeekNum']
                                        ,suffixes=('_max','_avg'))

DispenseMerge.insert(4,
                    'WeekOfDate',
                    DispenseMerge['WeekNum'].map(DateMap.to_dict()))

WeekSumMerge  = SumWeekUsage.merge(SumWeekDispense
                                        ,how='inner'
                                        ,on=['Name','Number','WeekNum']
                                        ,suffixes=('_Usage','_Dispense'))

WeekSumMerge.insert(3,
                    'WeekOfDate',
                    WeekSumMerge['WeekNum'].map(DateMap.to_dict()))




with pd.ExcelWriter(OutputFileName) as writer:
    raw.to_excel(writer, sheet_name='RAW', index=False)
    UsageByDay.to_excel(writer, sheet_name='Usage By Day', index=False)
    DispenseByDay.to_excel(writer, sheet_name='Dispense By Day', index=False)
    WeekSumMerge.to_excel(writer, sheet_name='Weekly Totals', index=False)
    UsageMerge.to_excel(writer, sheet_name='Weekly Usage Metrics', index=False, float_format='%.0f')
    DispenseMerge.to_excel(writer, sheet_name='Weekly Dispense Metrics', index=False, float_format='%.0f')
    
                    


C:\Users\brand\AppData\Local\Temp/ipykernel_19000/1237142610.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  raw.calendardate = pd.to_datetime(raw['CalendarDate'])


In [7]:
WeekSumMerge  = SumWeekUsage.merge(SumWeekDispense
                                        ,how='inner'
                                        ,on=['Name','Number','WeekNum']
                                        ,suffixes=('_Usage','_Dispense'))

WeekSumMerge.insert(3,
                    'WeekOfDate',
                    WeekSumMerge['WeekNum'].map(DateMap.to_dict()))


In [8]:
WeekSumMerge

,Name,Number,WeekNum,WeekOfDate,0.01c_Usage,0.05c_Usage,0.10c_Usage,0.25c_Usage,1.00n_Usage,5.00n_Usage,...,0.01c_Dispense,0.05c_Dispense,0.10c_Dispense,0.25c_Dispense,1.00n_Dispense,5.00n_Dispense,10.00n_Dispense,20.00n_Dispense,50.00n_Dispense,100.00n_Dispense
0,Sam's Club Liquor Box,4754,22,5/28/2022,-134,-17,-49,-100,-54,-19,...,-300,-120,-150,-164,-130,-64,0,-27,0,0
1,Sam's Club Liquor Box,4754,23,5/29/2022,-499,-110,-171,-304,-206,-37,...,-1800,-720,-900,-720,-475,-217,0,-115,0,0
2,Sam's Club Liquor Box,4754,24,6/10/2022,-180,-37,-81,-167,-239,-100,...,-1800,-720,-900,-740,-495,-257,0,-142,0,0
3,Sam's Club Liquor Box,4754,25,6/12/2022,-482,-129,-217,-361,-233,-88,...,-1800,-720,-900,-756,-480,-262,0,-133,0,0
4,Sam's Club Liquor Box,4754,26,6/19/2022,-360,-82,-137,-311,-291,-109,...,-1727,-675,-925,-776,-533,-271,0,-173,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,Walmart,7357,27,6/26/2022,-275,0,0,-55,555,253,...,-275,0,0,-55,-200,-86,0,-65,0,0
3590,Walmart,7357,28,7/3/2022,-500,-80,-50,-135,684,193,...,-500,-80,-50,-135,-214,-105,0,-65,0,0
3591,Walmart,7357,29,7/10/2022,-595,-120,-90,-105,810,148,...,-595,-120,-90,-105,-170,-110,0,-59,0,0
3592,Walmart,7357,30,7/17/2022,-311,0,0,-200,585,160,...,-311,0,0,-200,-263,-136,0,-62,0,0
